In [ ]:
import pandas as pd
import networkx as nx

In [ ]:
trips_filename = '../201508_trip_data.csv'
stations_filename = '../201508_station_data.csv'

trips_pdf = pd.read_csv(trips_filename)
stations_pdf = pd.read_csv(stations_filename)

In [ ]:
display(stations_pdf.head())

In [ ]:
# create nodes from stations.
#
# although networkx used hashed values for nodes, i.e. we don't need to
# have 'id' column, we decided to keep the same format as GraphFrame (PySpark).
#
stations_pdf = stations_pdf.rename(columns={'station_id':'id'})

In [ ]:
# index on the station id, now we call it 'id'
stations_pdf.set_index('id', inplace = True)
display(stations_pdf.head())

In [ ]:
nodes_dict = stations_pdf.to_dict('index')
#display(nodes_dict)
nodes_list = [x for x in nodes_dict.items()]

In [ ]:
#display(nodes_list)

In [ ]:
display(trips_pdf.head())

In [ ]:
# pick our set of columns to represent the edges
edge_columns = ['Start Terminal', 'End Terminal', 'Trip ID', 'Start Date', 'End Date']

trips_edges_pdf = trips_pdf.loc[:,edge_columns]

# rename the start and end terminal to 'src' and 'dst'
# Although not reqired we'll use the sample column names as the ones used with DataFrame 
# (PySpark) "src" (source vertex ID of edge) and "dst" (destination vertex ID of edge).

trips_edges_pdf = trips_edges_pdf.rename(columns={'Start Terminal':'src', 'End Terminal':'dst'})

In [ ]:
display(trips_edges_pdf.head())

In [ ]:
trips_edges_pdf.loc[0, 'src']

In [ ]:
edges_list = []
for i in range(trips_edges_pdf.shape[0]):
    edges_list.append((trips_edges_pdf.loc[i, 'src'], trips_edges_pdf.loc[i, 'dst'], 
                       {'Start Date': trips_edges_pdf.loc[i, 'Start Date'], 
                        'End Date': trips_edges_pdf.loc[i, 'End Date']}))

In [ ]:
edges_list[0]

In [ ]:
g = nx.MultiDiGraph()

In [ ]:
g.add_nodes_from(nodes_list)

In [ ]:
g.nodes[3]

In [ ]:
g.add_edges_from(edges_list)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# plt.subplot(121)
# nx.draw(g, with_labels=True, font_weight='bold')
# plt.show()

In [ ]:
x = g.in_degree()

In [ ]:
sorted(x, key=lambda x:x[1], reverse=True)